# TorchFort example train_distributed sd

Running on SDumont

Original file: /examples/fortran/simulation/train_distributed.f90

In [3]:
%cd /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation

/scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation


In [4]:
! singularity exec --nv /scratch/ampemi/eduardo.miranda2/torchfort.sif mpiexec --version

mpiexec (OpenRTE) 4.1.7a1

Report bugs to http://www.open-mpi.org/community/help/


In [ ]:
! singularity exec --nv /scratch/ampemi/eduardo.miranda2/torchfort.sif bash -c " \
mpirun --allow-run-as-root -np 2 ./train_distributed --simulation_device -1 --train_device -1 "

 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: cpu
 	train_device: cpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable TORCHFORT_LOGDIR was not specified. To enable logging, set TORCHFORT_LOGDIR to a writeable directory prior to launching wandb_helper.py and your torchfort enabled application.
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable TORCHFORT_LOGDIR was not specified. To enable logging, set TORCHFORT_LOGDIR to a writeable directory prior to launching wandb_helper.py and your torchfort enabled application.
 start training...
TORCHFORT::INFO: model: mymodel, step_train: 100, loss: 0.311064, lr: 0.000999998
TORCHFORT::INFO: model: mymodel, step_train: 200, loss: 5.69758, lr: 0.00099999
TORCHFORT::INFO: model: mymodel, step_train: 3

In [ ]:
%%writefile train_distributed.srm
#!/bin/bash
#SBATCH --partition=sequana_cpu_dev   # SLURM_JOB_PARTITION
#SBATCH --job-name=train_distributed  # SLURM_JOB_NAME
#SBATCH --nodes=1                     # SLURM_JOB_NUM_NODES
#SBATCH --ntasks-per-node=2           # SLURM_NTASKS_PER_NODE
#SBATCH --cpus-per-task=1             # SLURM_CPUS_PER_TASK
#SBATCH --distribution=block:cyclic   # SLURM_DISTRIBUTION
#SBATCH --time=00:10:00               # Limit execution time
# SLURM output environment variables:
# https://slurm.schedmd.com/sbatch.html#SECTION_OUTPUT-ENVIRONMENT-VARIABLES
echo '======================================='
echo '- Job ID:' $SLURM_JOB_ID
echo '- Nº of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- Nº of tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- Nº of tasks:' $SLURM_NTASKS
echo '- Nº of cpus per task:' $SLURM_CPUS_PER_TASK
echo '- Partition:' $SLURM_JOB_PARTITION
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
echo -n '- Nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST
echo '----------------------------------------'
#cd $SLURM_SUBMIT_DIR
cd /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation
#export SLURM_MPI_TYPE=pmi2
#export OMPI_HOME=$(dirname $(dirname $(which mpirun)))
#module load openmpi/gnu/4.1.4+cuda-11.2_sequana
echo -n '<1. starting python script > ' && date
echo '-- output -----------------------------'
#srun --cpu_bind=cores singularity exec torchfort.sif mpirun -np 4 ./sum_mpi
#srun --mpi=pmi2 -n 4 hostname
#srun singularity exec \
#    --bind /usr/lib64/mpi:/usr/lib64/mpi \
#    torchfort.sif ./sum_mpi
#srun --mpi=none singularity exec torchfort.sif mpirun --allow-run-as-root ./sum_mpi
#srun singularity exec --nv --bind /usr/lib64:/host_libs \
#    torchfort.sif bash -c "export LD_LIBRARY_PATH=/host_libs:\$LD_LIBRARY_PATH && ./sum_mpi"

srun \
singularity exec --nv /scratch/ampemi/eduardo.miranda2/torchfort.sif bash -c " \
mpirun --allow-run-as-root ./train_distributed --simulation_device -1 --train_device -1 "

echo '-- end --------------------------------'
echo -n '<2. quit>                    ' && date